In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install torch torchvision
!pip install efficientnet_pytorch

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 1. Hyperparameters
num_classes = 4  # Set this to the number of classes in your dataset
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# 2. Data Transforms and Dataloaders for Training and Validation
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Input size for EfficientNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/RetinalOCT_Dataset/train', transform=transform)
valid_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/RetinalOCT_Dataset/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 3. Model, Loss, Optimizer
model = models.efficientnet_b0(pretrained=True)  # Using EfficientNet-B0
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)  # Adjust the final layer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 4. Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, preds = torch.max(outputs, 1)
        correct_train += torch.sum(preds == labels.data)
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_train.double() / total_train

    # Validation phase
    model.eval()
    valid_loss = 0.0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_valid += torch.sum(preds == labels.data)
            total_valid += labels.size(0)

    epoch_valid_loss = valid_loss / len(valid_loader)
    epoch_valid_accuracy = correct_valid.double() / total_valid

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Validation Loss: {epoch_valid_loss:.4f}, '
          f'Training Accuracy: {epoch_accuracy:.4f}, '
          f'Validation Accuracy: {epoch_valid_accuracy:.4f}')

    # Save the model if it has the best validation accuracy so far
    if epoch_valid_accuracy > best_accuracy:
        best_accuracy = epoch_valid_accuracy
        torch.save(model.state_dict(), '/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_best_model.pth')


Epoch 1/10, Train Loss: 0.6301, Validation Loss: 0.3158, Training Accuracy: 0.7720, Validation Accuracy: 0.8850
Epoch 2/10, Train Loss: 0.3318, Validation Loss: 0.4279, Training Accuracy: 0.8905, Validation Accuracy: 0.8950
Epoch 3/10, Train Loss: 0.2415, Validation Loss: 0.3082, Training Accuracy: 0.9145, Validation Accuracy: 0.9000
Epoch 4/10, Train Loss: 0.1835, Validation Loss: 0.3511, Training Accuracy: 0.9340, Validation Accuracy: 0.9125
Epoch 5/10, Train Loss: 0.0963, Validation Loss: 0.4080, Training Accuracy: 0.9690, Validation Accuracy: 0.8975
Epoch 6/10, Train Loss: 0.1260, Validation Loss: 0.3383, Training Accuracy: 0.9585, Validation Accuracy: 0.9100
Epoch 7/10, Train Loss: 0.0910, Validation Loss: 0.2561, Training Accuracy: 0.9710, Validation Accuracy: 0.9325
Epoch 8/10, Train Loss: 0.0677, Validation Loss: 0.3057, Training Accuracy: 0.9760, Validation Accuracy: 0.9050
Epoch 9/10, Train Loss: 0.0641, Validation Loss: 0.2584, Training Accuracy: 0.9775, Validation Accuracy:

In [14]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 1. Hyperparameters
num_classes = 4  # Set this to the number of classes in your dataset
batch_size = 32

# 2. Data Transforms and Dataloader for Testing
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Input size for EfficientNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/RetinalOCT_Dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 3. Model
model = models.efficientnet_b0(pretrained=False)  # Load the same model architecture
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)  # Adjust the final layer

# Load the best model weights
model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_best_model.pth'))
model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model.eval()

# 4. Testing Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += torch.sum(preds == labels.data)

test_accuracy = 100 * correct.double() / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


<ipython-input-14-585c2be47c50>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_be

Test Accuracy: 87.75%


In [15]:
# Load the saved model
model = models.efficientnet_b0(pretrained=False)
# Change num_classes to 4 to match the saved model
num_classes = 4
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_best_model.pth', map_location=device))
model = model.to(device)

<ipython-input-15-caed78cc1b9a>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_bes

In [16]:
from PIL import Image

# Function to make predictions on a single image
def predict_image(image_path, model, transform, device):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB') # Convert the image to RGB
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    # Set the model to evaluation mode
    model.eval()

    # No need to compute gradients during prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return predicted.item()

# Load the saved model
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_best_model.pth', map_location=device))
model = model.to(device)

# Example usage
image_path = '/content/drive/MyDrive/RetinalOCT_Dataset/test/NORMAL/normal_test_1005.jpg'
predicted_class = predict_image(image_path, model, transform, device)
print(f'Predicted class: {predicted_class}')

Predicted class: 3


<ipython-input-16-f63a0f0aa092>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_be

In [18]:
import numpy as np
import torch  # Import PyTorch instead of TensorFlow
from torchvision import transforms, models
from PIL import Image

# Load the trained model using PyTorch
model = models.efficientnet_b0(pretrained=False)
num_classes = 4  # Set this to the number of classes in your dataset
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)  # Adjust the final layer
model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_best_model.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # Set the model to evaluation mode


def predict_eye_disease(image_path):
    # Load and preprocess the image using PyTorch transforms
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to the input size expected by EfficientNet
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Make prediction
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted_class = torch.max(outputs, 1)

    # Map the prediction to class names
    class_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']  # Update according to your dataset classes
    result = class_names[predicted_class.item()]

    return result

# Example usage
image_path = "/content/drive/MyDrive/RetinalOCT_Dataset/train/CNV/cnv_train_1005.jpg"
prediction = predict_eye_disease(image_path)
print(f"The predicted class is: {prediction}")

<ipython-input-18-51daf7df1645>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/RetinalOCT_Dataset/efficientnet_be

The predicted class is: CNV
